<a href="https://colab.research.google.com/github/lizbiz0510/CORNEJO_A3/blob/main/Cornejo_A3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install any missing dependencies
! pip install pandasql


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=e73623eb5fe07a24a1e484e32c46d728355c2f54dddbd3ccce1d230c0a8d02d9
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql


In [95]:

import pandas as pd
import pandasql as ps

with open('secondary_data.csv') as f1:
  with open('data.csv', 'w') as f2:
    f2.write(f1.read().replace(';', ',').replace('-', '_').replace(" ", ""))

data = pd.read_csv('data.csv')
print(data.head().to_string()) 
print(data.columns)


  class  cap_diameter cap_shape cap_surface cap_color does_bruise_or_bleed gill_attachment gill_spacing gill_color  stem_height  stem_width stem_root stem_surface stem_color veil_type veil_color has_ring ring_type spore_print_color habitat season
0     p         15.26         x           g         o                    f               e          NaN          w        16.95       17.09         s            y          w         u          w        t         g               NaN       d      w
1     p         16.60         x           g         o                    f               e          NaN          w        17.99       18.19         s            y          w         u          w        t         g               NaN       d      u
2     p         14.07         x           g         o                    f               e          NaN          w        17.80       17.74         s            y          w         u          w        t         g               NaN       d      w
3     p     

In [96]:
trainset = data.sample(frac=0.75)
testset = data.drop(trainset.index)
print('Length of Trainset:', len(trainset),'\nLength of Testset:', len(testset))

Length of Trainset: 45802 
Length of Testset: 15267


In [99]:
def checkCategoricalVariable(varName, x, data):
  sql_query = "SELECT * FROM data WHERE " + str(varName) + " = '" + str(x[str(varName)]) + "'"
  temp = data
  data = ps.sqldf(sql_query,locals())
  if len(data) < 1:
    sql_query = "SELECT * FROM temp WHERE class = 'p'"
    df1 = ps.sqldf(sql_query,locals())
    sql_query = "SELECT * FROM temp WHERE class = 'e'"
    df2 = ps.sqldf(sql_query,locals())
    if len(df1) > len(df2):
      return df1.iloc[0]
    return df2.iloc[0]
  return data

def checkNumericVariable(varName, x, data, _range):
  if x[str(varName)] >= _range[0] and x[str(varName)] < _range[1]:
    sql_query = "SELECT * FROM data WHERE " + str(varName) + "  >= " + str(_range[0]) + " AND " + str(varName) + "  < " + str(_range[1])
    temp = data
    data = ps.sqldf(sql_query,locals())
    if len(data) < 1:
      sql_query = "SELECT * FROM temp WHERE class = 'p'"
      df1 = ps.sqldf(sql_query,locals())
      sql_query = "SELECT * FROM temp WHERE class = 'e'"
      df2 = ps.sqldf(sql_query,locals())
      if len(df1) > len(df2):
        return df1.iloc[0]
      return df2.iloc[0]
    return data
  return data

In [131]:
def classify(x, trainset):
  categorical_vars = ["cap_shape", "cap_surface", "cap_color", "does_bruise_or_bleed", "gill_attachment",
                      "gill_color", "stem_root", "stem_surface", "stem_color", "veil_type", 
                      "veil_color", "has_ring", "ring_type", "habitat", "season"]
  data = trainset

  while len(data) > 1 and len(categorical_vars) > 0:
    data = checkCategoricalVariable(categorical_vars.pop(0), x, data)

  if len(data) == 1:
    return data.iloc[0]['class']

  numerical_vars = {"cap_diameter": 60, "stem_height": 30, "stem_width": 100}

  for var in numerical_vars:
    for i in range(0, numerical_vars[var], 10):
      data = checkNumericVariable(var, x, data, (i, i + 10))
      if len(data) == 1:
        return data.iloc[0]['class']
  
  sql_query = "SELECT * FROM data WHERE class = 'p'"
  df1 = ps.sqldf(sql_query,locals())
  sql_query = "SELECT * FROM data WHERE class = 'e'"
  df2 = ps.sqldf(sql_query,locals())
  if len(df1) > len(df2):
    return df1.iloc[0]['class']
  return df2.iloc[0]['class']

print(testset.iloc[90])
 

class                       p
cap_diameter             7.44
cap_shape                   p
cap_surface                 g
cap_color                   n
does_bruise_or_bleed        f
gill_attachment             e
gill_spacing              NaN
gill_color                  w
stem_height              8.58
stem_width              15.59
stem_root                 NaN
stem_surface                y
stem_color                  w
veil_type                   u
veil_color                  w
has_ring                    t
ring_type                   p
spore_print_color         NaN
habitat                     d
season                      a
Name: 394, dtype: object


In [134]:
sampleset = testset[:5]
testset = testset[5:]

def evaluate(inputs,data):
  correct = 0 
  for i in range(50):
    if inputs.iloc[i]["class"] == classify(inputs.iloc[i], data):
      correct +=1
  return correct/50
print(evaluate(testset,trainset))


1.0


In [135]:
print("The decision tree classifies")
print(sampleset.iloc[0])
print("as " + ("not poisonous" if classify(sampleset.iloc[0], trainset).strip()=='e' else "poisonous"))
print('_____________________________________________________\n')

print("The decision tree classifies")
print(sampleset.iloc[1])
print("as " + ("not poisonous" if classify(sampleset.iloc[1], trainset).strip()=='e' else "poisonous"))
print('_____________________________________________________\n')

print("The decision tree classifies")
print(sampleset.iloc[2])
print("as " + ("not poisonous" if classify(sampleset.iloc[2], trainset).strip()=='e' else "poisonous"))
print('_____________________________________________________\n')

print("The decision tree classifies")
print(sampleset.iloc[3])
print("as " + ("not poisonous" if classify(sampleset.iloc[3], trainset).strip()=='e' else "poisonous"))
print('_____________________________________________________\n')

print("The decision tree classifies")
print(sampleset.iloc[4])
print("as " + ("not poisonous" if classify(sampleset.iloc[4], trainset).strip()=='e' else "poisonous"))
print('_____________________________________________________\n')

The decision tree classifies
class                       p
cap_diameter            12.27
cap_shape                   f
cap_surface                 g
cap_color                   o
does_bruise_or_bleed        f
gill_attachment             e
gill_spacing              NaN
gill_color                  w
stem_height             16.15
stem_width              17.17
stem_root                   s
stem_surface                y
stem_color                  w
veil_type                   u
veil_color                  w
has_ring                    t
ring_type                   g
spore_print_color         NaN
habitat                     d
season                      w
Name: 47, dtype: object
as poisonous
_____________________________________________________

The decision tree classifies
class                       p
cap_diameter            16.03
cap_shape                   f
cap_surface                 g
cap_color                   o
does_bruise_or_bleed        f
gill_attachment             e
gill_spaci